<a href="https://colab.research.google.com/github/Raj0978/Parcil_Safety/blob/main/Customer_Survey_Analysis(Topic_Modeling_%26_Sentiment_Analysis).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 19.6 MB/s eta 0:00:00


In [ ]:
pip install dash

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 38.5 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_md -qq


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 11.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.language import Language
from spacy.pipeline import EntityRuler
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
import warnings
import re
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output



In [ ]:
warnings.filterwarnings('ignore')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:


comments=pd.read_csv('Post_Purchase.csv')

In [ ]:
comments.head(20)

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,What motivated you to purchase from us today? (This helps us a lot - thanks in advance!),What ONE factor influenced your decision to purchase the most?,"Did you have any concerns, doubts, or uncertainties that almost stopped you from purchasing today? If so, what were they? (We love feedback - don’t hold back)","If you were considering another brand, who were they and what led you to choosing us instead?"
0,Post Purchase Survey V1,rickd28@yahoo.com,Richard,Dougherty,2024-01-27T00:52:41.975Z,post-checkout,593.96,I want to be prepared for any emergencies and ...,Other (please specify) (Black_Scout_Survial_Tr...,NaN,NaN
1,Post Purchase Survey V1,josephdhorn@gmail.com,Joseph,Horn,2024-01-27T00:57:53.335Z,post-checkout,227.10,I want to be prepared for any emergencies and ...,Product description of benefits/features,My main concern is if the respiratory mask wil...,"The mask looks great, and it protects from vir..."
2,Post Purchase Survey V1,gypsydrifter.mh@gmail.com,Albert,hunter,2024-01-27T01:17:49.725Z,post-checkout,378.47,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,No reviews,Nope
3,Post Purchase Survey V1,bhemerson1048@yahoo.com,Brandon,Emerson,2024-01-27T01:58:30.761Z,post-checkout,497.30,I want to be prepared for any emergencies and ...,Product description of benefits/features,No,Mira
4,Post Purchase Survey V1,eric.o.hansen5@gmail.com,Eric,Hansen,2024-01-27T02:58:35.183Z,post-checkout,262.54,I want to be prepared for any emergencies and ...,Product reviews and testimonials,NaN,NaN
5,Post Purchase Survey V1,jordanweil159@yahoo.com,Jordan,weil,2024-01-27T04:02:40.003Z,post-checkout,326.69,My job requires protection against environment...,Product reviews and testimonials,NaN,Mira. You guys are more affordable.
6,Post Purchase Survey V1,peter.d.young2020@gmail.com,Peter,Young,2024-01-27T04:06:34.004Z,post-checkout,515.92,I want to be prepared for any emergencies and ...,Product reviews and testimonials,NaN,NaN
7,Post Purchase Survey V1,chris@jescoboatcenter.com,chris,Parrott,2024-01-27T04:19:32.406Z,post-checkout,375.33,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,"More information on fit, forum sites say your ...","Mira, price"
8,Post Purchase Survey V1,mcapotrio96@verizon.net,Michael,Capotrio,2024-01-27T04:23:42.695Z,post-checkout,248.65,I want to be prepared for any emergencies and ...,Other (please specify) (Price),NaN,NaN
9,Post Purchase Survey V1,pdt1043@aol.com,Michael,Apple,2024-01-27T13:32:20.047Z,post-checkout,277.56,My job requires protection against environment...,Engaging product images and videos on website,NaN,NaN


In [ ]:
comments.shape

(1488, 11)

In [ ]:
comments.rename(columns={'If you were considering another brand, who were they and what led you to choosing us instead?': 'Competitor',
                         'Did you have any concerns, doubts, or uncertainties that almost stopped you from purchasing today? If so, what were they? (We love feedback - don’t hold back)': 'Concern',
                         'What ONE factor influenced your decision to purchase the most?': 'Purchase_Influencing_Factor',
                         'What motivated you to purchase from us today? (This helps us a lot - thanks in advance!)':'Purchase_Reason'},inplace=True)

In [ ]:
comments.columns

Index(['surveyName', 'email', 'customerFirstName', 'customerLastName',
       'responseDate', 'channelName', 'orderValue', 'Purchase_Reason',
       'Purchase_Influencing_Factor', 'Concern', 'Competitor'],
      dtype='object')

In [ ]:
comments['Competitor'].isnull().sum()

837

In [ ]:
comments=comments.dropna(subset=['Competitor'])

In [ ]:
nlp = spacy.load("en_core_web_sm")


In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)


In [ ]:
# Define custom patterns to match variations of organization names
custom_phrases = [
    "3M", "3m","m3","Honeywell", "MSA Safety", "Moldex", "North By Honeywell",
    "GVS Group", "Draeger Safety", "Uvex", "Mira", "Avon Protection Systems",
    "Scott Safety", "RSG Safety", "Sundstorm Safety",
    "MIRA", "mira","MSA "," MSA"
]
# Define additional NER categories
better_price_phrases = [
    "Price", "Cost", "Expensive", "Cheaper", "Affordable", "Discount", "Deal", "50% off","price","cost","expensive",
    "cheaper","affordable","discount"
]

feature_phrases = [
    "Looks great", "protects", "protection", "individuality", "Bulk Canisters", "design",
    "ease of filter connection", "bundle", "filters", "no-fog", "anti-fogging", "field of view",
    "airtight seal", "filter", "bundle", "safe", "safer", "safest", "esthetics", "style",
    "fiberglass", "silicone", "fit", "fitting", "size", "mirrored lenses", "comfort",
    "comfortable", "reputation", "full face", "colour", "Colour", "combo", "Combo", "look",
    "variety", "tinted shield", "field of vision", "unique", "Unique", "shelf life","Bundle","Protects",
    "Protection","Individuality","Filters","Filter","Size","Fit","Fitting","Full face","Full Face","Full-Face","full-face",
    "Quality","quality"
]

advertising_phrases = [
    "Advertising", "videos", "website", "IG", "youtube", "Youtube", "ad",
    "instagram", "Instagram", "TikTok", "Facebook", "site", "marketing", "information"
]



In [ ]:
# Add the custom phrases to the PhraseMatcher
patterns = [nlp.make_doc(text) for text in custom_phrases]
phrase_matcher.add("ORG_CUSTOM", None, *patterns)
better_price_patterns = [nlp.make_doc(text) for text in better_price_phrases]
phrase_matcher.add("BETTER_PRICE", None, *better_price_patterns)
quality_patterns = [nlp.make_doc(text) for text in feature_phrases]
phrase_matcher.add("FEATURE", None, *quality_patterns)
advertising_patterns = [nlp.make_doc(text) for text in advertising_phrases]
phrase_matcher.add("ADVERTISING", None, *advertising_patterns)

In [ ]:
# # Define a function to handle matches and include both default and custom NERs
# Define a function to handle matches and include both default and custom NERs
@Language.component("custom_ner_component")
def add_custom_ents(doc):
    matches = phrase_matcher(doc)
    spans = []
    for match_id, start, end in matches:
        match_label = nlp.vocab.strings[match_id]
        if match_label == "ORG_CUSTOM":
            span = Span(doc, start, end, label="ORG")
        else:
            span = Span(doc, start, end, label=match_label)
        spans.append(span)

    # Combine with existing entities ensuring no overlap
    existing_ents = list(doc.ents)
    for span in spans:
        if not any(span.start < ent.end and span.end > ent.start for ent in existing_ents):
            existing_ents.append(span)

    # Sort the entities to maintain proper sequence
    doc.ents = sorted(existing_ents, key=lambda ent: ent.start)
    return doc

In [ ]:
# def setup_custom_ner(nlp):
#     if "custom_ner" not in nlp.pipe_names:
#         @spacy.Language.component("custom_ner")
#         def custom_ner_component(doc):
#             spans = list(doc.ents)

#             for match in custom_phrases:
#                 regex = re.compile(r'\b{}\b'.format(re.escape(match)), re.IGNORECASE)
#                 for match in regex.finditer(doc.text):
#                     start, end = match.span()
#                     span = Span(doc, start, end, label="ORG")
#                     spans.append(span)

#             spans = spacy.util.filter_spans(spans)
#             doc.ents = spans
#             return doc

#         nlp.add_pipe("custom_ner", last=True)


In [ ]:
# # Process a text
# text = "3M, Honeywell, MSA Safety, Moldex, North By Honeywell, GVS Group, Draeger Safety, Uvex, Mira, Avon Protection Systems, Scott Safety, RSG Safety, Sundstorm Safety"
# doc = nlp(text)

In [ ]:
nlp.add_pipe("custom_ner_component", after='ner')

<function __main__.add_custom_ents(doc)>

In [ ]:
comments=comments.dropna(subset=['Competitor'])

In [ ]:
# Function to find NERs in the competitors column and classify them
def find_ners(text):
    doc = nlp(text)
    orgs, better_price, Feature, advertising = [], [], [], []
    for ent in doc.ents:
        if ent.label_ == "CARDINAL" and ent.text == "3":
            orgs.append(("3M", "ORG"))
        else:
            if ent.label_ == "ORG":
                orgs.append((ent.text, ent.label_))
            elif ent.label_ == "BETTER_PRICE":
                better_price.append((ent.text, ent.label_))
            elif ent.label_ == "FEATURE":
                Feature.append((ent.text, ent.label_))
            elif ent.label_ == "ADVERTISING":
                advertising.append((ent.text, ent.label_))
    return orgs, better_price, Feature, advertising


In [ ]:
# Apply the function to the competitors column
comments["org_entities"], comments["better_price_entities"], comments["quality_entities"], comments["advertising_entities"] = zip(*comments["Competitor"].apply(find_ners))

In [ ]:
comments[comments['org_entities'].apply(len)==0]

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,Purchase_Reason,Purchase_Influencing_Factor,Concern,Competitor,org_entities,better_price_entities,quality_entities,advertising_entities
1,Post Purchase Survey V1,josephdhorn@gmail.com,Joseph,Horn,2024-01-27T00:57:53.335Z,post-checkout,227.10,I want to be prepared for any emergencies and ...,Product description of benefits/features,My main concern is if the respiratory mask wil...,"The mask looks great, and it protects from vir...",[],[],"[(protects, FEATURE)]",[]
2,Post Purchase Survey V1,gypsydrifter.mh@gmail.com,Albert,hunter,2024-01-27T01:17:49.725Z,post-checkout,378.47,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,No reviews,Nope,[],[],[],[]
12,Post Purchase Survey V1,pete.graziano3@gmail.com,Peter,Graziano,2024-01-27T16:07:25.423Z,post-checkout,248.59,My job requires protection against environment...,Product description of benefits/features,None at the moment. I check back in.,"Price, and individuality",[],"[(Price, BETTER_PRICE)]","[(individuality, FEATURE)]",[]
13,Post Purchase Survey V1,deputyfelix@gmail.com,Robert,Felix,2024-01-27T17:04:09.046Z,post-checkout,238.12,I want to be prepared for any emergencies and ...,Other (please specify) (First responder discou...,No,"MiRA was more expensive, although design was m...",[],"[(expensive, BETTER_PRICE)]","[(design, FEATURE)]",[]
18,Post Purchase Survey V1,christophermeegan84@gmail.com,Christopher,Meegan,2024-01-27T23:14:03.106Z,post-checkout,458.95,Other (please specify) (Life is dangerous and ...,Other (please specify) (Black Scout Survival),Price but I found a fantastic coupon code that...,Quality and price are some of the best around ...,[],"[(price, BETTER_PRICE)]","[(Quality, FEATURE)]",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467,Post Purchase Survey V1,tschultz227@yahoo.com,Tiffany,Jenkins,2024-06-05T01:49:25.462Z,post-checkout,73.47,My job requires protection against environment...,Other (please specify) (Used this filter befor...,"No, I'm just glad I was able to pre-order them.",No other brand,[],[],[],[]
1474,Post Purchase Survey V1,miketice17@gmail.com,Michael,Tice,2024-06-06T21:37:13.509Z,post-checkout,160.78,I want to be prepared for any emergencies and ...,Product reviews and testimonials,price,no,[],[],[],[]
1475,Post Purchase Survey V1,laswellmicheal@gmail.com,Micheal,Laswell,2024-06-07T01:07:47.582Z,post-checkout,161.97,My job requires protection against environment...,Product reviews and testimonials,No,No,[],[],[],[]
1481,Post Purchase Survey V1,wallenburton@yahoo.com,Burton,Wallen,2024-06-08T01:59:12.813Z,post-checkout,567.14,I want to be prepared for any emergencies and ...,Product reviews and testimonials,NaN,Your product appears to have superior material...,[],[],[],[]


In [ ]:
comments[comments['Competitor'].str.contains('3M')]

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,Purchase_Reason,Purchase_Influencing_Factor,Concern,Competitor,org_entities,better_price_entities,quality_entities,advertising_entities
14,Post Purchase Survey V1,info@builtcreative.com,Shane,Poteete,2024-01-27T17:09:35.317Z,post-checkout,560.58,I want to be prepared for any emergencies and ...,Other (please specify) (Promotion from 704 Tac...,"No. 704 Tactical does reviews of products, and...",Other brands were 3M or MIRA. Discount/price w...,"[(3M, ORG), (MIRA, ORG)]","[(Discount, BETTER_PRICE), (price, BETTER_PRICE)]",[],[]
30,Post Purchase Survey V1,dcmoose@hawaii.rr.com,DOUGLAS,MOOSE,2024-01-29T02:24:12.846Z,post-checkout,72.77,I am a hobbyist or DIY enthusiast in areas nee...,Product description of benefits/features,Would like to find out if the tinted face shie...,I have a 3M half-face mask (2nd I've owned). ...,"[(3M, ORG), (3M, ORG)]","[(discount, BETTER_PRICE)]","[(full-face, FEATURE), (fit, FEATURE), (no-fog...",[]
57,Post Purchase Survey V1,bill.kraemer@gmail.com,Bill,Kraemer,2024-01-31T19:17:18.077Z,post-checkout,19.97,I am a hobbyist or DIY enthusiast in areas nee...,Other (please specify),Shipping insurance. You should own that.,3M. \n\nProduct specificity,[],[],[],[]
216,Post Purchase Survey V1,paint@roecustompainting.com,Rick,Roe,2024-02-18T22:51:56.196Z,post-checkout,204.20,My job requires protection against environment...,Product description of benefits/features,NaN,"3M, Cost","[(3M, ORG)]","[(Cost, BETTER_PRICE)]",[],[]
249,Post Purchase Survey V1,dotmatrixdisplay@yahoo.com,Sigmund,Vaccaro,2024-02-23T14:25:07.316Z,post-checkout,172.85,My job requires protection against environment...,Product reviews and testimonials,I was not sure if you were a real company or j...,The price point is good but am yet to asses th...,"[(3M, ORG)]","[(price, BETTER_PRICE)]","[(quality, FEATURE), (quality, FEATURE)]",[]
272,Post Purchase Survey V1,nthornton28@icloud.com,Nicholas,Thornton,2024-02-26T14:11:54.892Z,post-checkout,122.97,I am a hobbyist or DIY enthusiast in areas nee...,Product description of benefits/features,I wasn’t sure at first if this was good for re...,3M . Good price and return policy and good rev...,"[(3M, ORG)]","[(price, BETTER_PRICE)]",[],[]
275,Post Purchase Survey V1,cos@hartfinefurniture.com,Cornelius,Arnett,2024-02-26T16:41:02.116Z,post-checkout,39.12,My job requires protection against environment...,Product reviews and testimonials,NaN,"I've used 3M respirators for years, your produ...","[(3M, ORG), (3M, ORG)]","[(discount, BETTER_PRICE)]",[],[]
291,Post Purchase Survey V1,kassondsk@aol.com,DEBRA,S. KASSON,2024-02-26T23:14:20.292Z,post-checkout,61.49,I am a hobbyist or DIY enthusiast in areas nee...,Other (please specify) (Commercial on tv 40% o...,No,3M. Both brands were recommended by different...,"[(Amazon, ORG), (3M, ORG)]",[],[],"[(website, ADVERTISING)]"
331,Post Purchase Survey V1,alex47771@gmail.com,Alex,Perez,2024-02-28T19:49:57.086Z,post-checkout,85.60,I am a hobbyist or DIY enthusiast in areas nee...,Product description of benefits/features,NaN,"3M, ad on Instagram with sale","[(3M, ORG), (Instagram, ORG)]",[],[],"[(ad, ADVERTISING)]"
341,Post Purchase Survey V1,tracker.rick82@gmail.com,Richard,Bain IV,2024-02-28T21:52:51.781Z,post-checkout,88.94,My job requires protection against environment...,Smooth website and navigation experience,Need to add size descriptions for the full fac...,3M. Price point,[],"[(Price, BETTER_PRICE)]",[],[]


In [ ]:
# Function to count occurrences of ORG entities and create breakdown
from collections import Counter
def count_entities(ner_list):
    entity_count = Counter()
    for entity, label in ner_list:
        # Standardize the organization name for ORG label
        if label == "ORG":
            entity = entity.lower().replace(" safety", "")
        if label == "CARDINAL" and entity == "3":
            entity_count["3M"] += 1
        else:
            entity_count[entity] += 1
    return dict(entity_count)


In [ ]:
# Aggregate the entity counts from all rows for each category
all_entity_counts = Counter()
all_better_price_counts = Counter()
all_quality_counts = Counter()
all_advertising_counts = Counter()

for entity_dict in comments["org_entities"].apply(count_entities):
    all_entity_counts.update(entity_dict)
for entity_dict in comments["better_price_entities"].apply(count_entities):
    all_better_price_counts.update(entity_dict)
for entity_dict in comments["quality_entities"].apply(count_entities):
    all_quality_counts.update(entity_dict)
for entity_dict in comments["advertising_entities"].apply(count_entities):
    all_advertising_counts.update(entity_dict)

In [ ]:
# Combine all counts into a single DataFrame
combined_counts_df = pd.DataFrame({
    "ORG": pd.Series(all_entity_counts),
    "BETTER_PRICE": pd.Series(all_better_price_counts),
    "QUALITY": pd.Series(all_quality_counts),
    "ADVERTISING": pd.Series(all_advertising_counts)
}).fillna(0).astype(int)

In [ ]:
combined_counts_df['BETTER_PRICE'].sum()

149

In [ ]:
combined_counts_df['ADVERTISING'].sum()

28

In [ ]:
combined_counts_df['QUALITY'].sum()

143

In [ ]:
Feature=combined_counts_df[combined_counts_df['QUALITY']>0]
Feature=Feature['QUALITY']
Feature=Feature.sort_values(ascending=True)
Feature=Feature.to_frame()

In [ ]:
Feature

,QUALITY
size,1
individuality,1
unique,1
safer,1
shelf life,1
field of vision,1
field of view,1
mirrored lenses,1
esthetics,1
no-fog,1


In [ ]:
comments.head(25)

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,Purchase_Reason,Purchase_Influencing_Factor,Concern,Competitor,org_entities,better_price_entities,quality_entities,advertising_entities
1,Post Purchase Survey V1,josephdhorn@gmail.com,Joseph,Horn,2024-01-27T00:57:53.335Z,post-checkout,227.10,I want to be prepared for any emergencies and ...,Product description of benefits/features,My main concern is if the respiratory mask wil...,"The mask looks great, and it protects from vir...",[],[],"[(protects, FEATURE)]",[]
2,Post Purchase Survey V1,gypsydrifter.mh@gmail.com,Albert,hunter,2024-01-27T01:17:49.725Z,post-checkout,378.47,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,No reviews,Nope,[],[],[],[]
3,Post Purchase Survey V1,bhemerson1048@yahoo.com,Brandon,Emerson,2024-01-27T01:58:30.761Z,post-checkout,497.30,I want to be prepared for any emergencies and ...,Product description of benefits/features,No,Mira,"[(Mira, ORG)]",[],[],[]
5,Post Purchase Survey V1,jordanweil159@yahoo.com,Jordan,weil,2024-01-27T04:02:40.003Z,post-checkout,326.69,My job requires protection against environment...,Product reviews and testimonials,NaN,Mira. You guys are more affordable.,"[(Mira, ORG)]","[(affordable, BETTER_PRICE)]",[],[]
7,Post Purchase Survey V1,chris@jescoboatcenter.com,chris,Parrott,2024-01-27T04:19:32.406Z,post-checkout,375.33,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,"More information on fit, forum sites say your ...","Mira, price","[(Mira, ORG)]","[(price, BETTER_PRICE)]",[],[]
12,Post Purchase Survey V1,pete.graziano3@gmail.com,Peter,Graziano,2024-01-27T16:07:25.423Z,post-checkout,248.59,My job requires protection against environment...,Product description of benefits/features,None at the moment. I check back in.,"Price, and individuality",[],"[(Price, BETTER_PRICE)]","[(individuality, FEATURE)]",[]
13,Post Purchase Survey V1,deputyfelix@gmail.com,Robert,Felix,2024-01-27T17:04:09.046Z,post-checkout,238.12,I want to be prepared for any emergencies and ...,Other (please specify) (First responder discou...,No,"MiRA was more expensive, although design was m...",[],"[(expensive, BETTER_PRICE)]","[(design, FEATURE)]",[]
14,Post Purchase Survey V1,info@builtcreative.com,Shane,Poteete,2024-01-27T17:09:35.317Z,post-checkout,560.58,I want to be prepared for any emergencies and ...,Other (please specify) (Promotion from 704 Tac...,"No. 704 Tactical does reviews of products, and...",Other brands were 3M or MIRA. Discount/price w...,"[(3M, ORG), (MIRA, ORG)]","[(Discount, BETTER_PRICE), (price, BETTER_PRICE)]",[],[]
18,Post Purchase Survey V1,christophermeegan84@gmail.com,Christopher,Meegan,2024-01-27T23:14:03.106Z,post-checkout,458.95,Other (please specify) (Life is dangerous and ...,Other (please specify) (Black Scout Survival),Price but I found a fantastic coupon code that...,Quality and price are some of the best around ...,[],"[(price, BETTER_PRICE)]","[(Quality, FEATURE)]",[]
20,Post Purchase Survey V1,williamladley440@gmail.com,William,Ladley,2024-01-28T00:07:43.211Z,post-checkout,229.66,I want to be prepared for any emergencies and ...,Product description of benefits/features,I’m a firefighter and was weary about putting ...,"MSA im familiar with this brand,ease of filter...","[(MSA, ORG)]",[],"[(ease of filter connection, FEATURE)]",[]


In [ ]:
filtered_comments = comments[comments["quality_entities"].apply(lambda ents: any(ent[0].lower() in ["Quality", "quality"] for ent in ents))]
filtered_comments

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,Purchase_Reason,Purchase_Influencing_Factor,Concern,Competitor,org_entities,better_price_entities,quality_entities,advertising_entities
18,Post Purchase Survey V1,christophermeegan84@gmail.com,Christopher,Meegan,2024-01-27T23:14:03.106Z,post-checkout,458.95,Other (please specify) (Life is dangerous and ...,Other (please specify) (Black Scout Survival),Price but I found a fantastic coupon code that...,Quality and price are some of the best around ...,[],"[(price, BETTER_PRICE)]","[(Quality, FEATURE)]",[]
47,Post Purchase Survey V1,rtkegley@gmail.com,RYAN,KEGLEY,2024-01-30T23:45:40.686Z,post-checkout,323.17,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,nope\n,Avon Protection & Mira Safety. Balance of qual...,"[(Avon Protection & Mira Safety, ORG)]","[(discount, BETTER_PRICE)]","[(quality, FEATURE)]",[]
140,Post Purchase Survey V1,75chair@gmail.com,Bev,Kelley-Miller,2024-02-08T02:46:16.709Z,post-checkout,40.47,I am a hobbyist or DIY enthusiast in areas nee...,Product reviews and testimonials,I wanted to get a quality mask without paying ...,I wanted a quality mask that protects my lungs...,[],[],"[(quality, FEATURE), (protects, FEATURE)]",[]
249,Post Purchase Survey V1,dotmatrixdisplay@yahoo.com,Sigmund,Vaccaro,2024-02-23T14:25:07.316Z,post-checkout,172.85,My job requires protection against environment...,Product reviews and testimonials,I was not sure if you were a real company or j...,The price point is good but am yet to asses th...,"[(3M, ORG)]","[(price, BETTER_PRICE)]","[(quality, FEATURE), (quality, FEATURE)]",[]
263,Post Purchase Survey V1,fairluka@gmail.com,Alicia,Daniells,2024-02-25T23:58:26.357Z,post-checkout,204.47,I am a hobbyist or DIY enthusiast in areas nee...,Other (please specify) (https://www.youtube.co...,Cost is quite high but my health is important ...,"No particular brand, just wanted something mor...",[],"[(affordable, BETTER_PRICE)]","[(quality, FEATURE)]",[]
307,Post Purchase Survey V1,sledshine@gmail.com,Jeff,Black,2024-02-27T17:16:49.547Z,post-checkout,104.22,My job requires protection against environment...,Other (please specify) (40% off sale),NaN,Quality,[],[],"[(Quality, FEATURE)]",[]
463,Post Purchase Survey V1,mark.g@twc.com,Mark,Gliessner,2024-03-03T17:53:48.415Z,post-checkout,130.15,I want to be prepared for any emergencies and ...,Product reviews and testimonials,NaN,"Mira, quality and value","[(Mira, ORG)]",[],"[(quality, FEATURE)]",[]
516,Post Purchase Survey V1,jfrollercoasterrider@gmail.com,Jonathan,Ford,2024-03-05T14:21:51.733Z,post-checkout,1070.12,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,No I don't have any concerns. I'm happy to sup...,Only Parcil Safety was an option for me. I'm t...,"[(Company, ORG)]",[],"[(quality, FEATURE)]",[]
659,Post Purchase Survey V1,batucke1128@gmail.com,Brent,Tucke,2024-03-18T22:48:35.951Z,post-checkout,85.78,My job requires protection against environment...,Other (please specify) (Mirrored finish on res...,NaN,I considered 3M but for the price and the qual...,"[(3M, ORG)]","[(price, BETTER_PRICE)]","[(quality, FEATURE)]",[]
664,Post Purchase Survey V1,mdbacon339@gmail.com,Matthew,Bacon,2024-03-19T12:58:58.915Z,post-checkout,22.02,I want to be prepared for any emergencies and ...,Engaging product images and videos on website,"Mask use in fire fighting situations, you don’...","3M, price and quality for the price made me c...","[(3M, ORG)]","[(price, BETTER_PRICE), (price, BETTER_PRICE)]","[(quality, FEATURE)]",[]


In [ ]:
comments["Purchase_Influencing_Factor"] = comments["Purchase_Influencing_Factor"].apply(lambda x: re.sub(r'^Other \(please specify\) ', '', x).strip('()'))


In [ ]:
PR=comments['Purchase_Reason'].value_counts()

In [ ]:
PR_top_3=PR.head(3)

In [ ]:
PR_top_3

Purchase_Reason
I want to be prepared for any emergencies and ensure safety for friends/family    326
My job requires protection against environmental hazards or harmful particles     157
I am a hobbyist or DIY enthusiast in areas needing respiratory protection.        127
Name: count, dtype: int64

In [ ]:
PIF=comments['Purchase_Influencing_Factor'].value_counts()

In [ ]:
PIF_Top_5=PIF.head(5)

In [ ]:
PIF_Top_5

Purchase_Influencing_Factor
Product description of benefits/features          255
Product reviews and testimonials                  183
Smooth website and navigation experience           57
Engaging product images and videos on website      44
Other (please specify                              10
Name: count, dtype: int64

In [ ]:
PIF

Purchase_Influencing_Factor
Product description of benefits/features                           255
Product reviews and testimonials                                   183
Smooth website and navigation experience                            57
Engaging product images and videos on website                       44
Other (please specify                                               10
                                                                  ... 
Instagram                                                            1
Saw it on Facebook and liked the look and reviews of the mask.       1
Sale Price                                                           1
Sale                                                                 1
Used this filter before and it's great                               1
Name: count, Length: 104, dtype: int64

In [ ]:
# Aggregate the entity counts from all rows
# Combine all counts into a single DataFrame
combined_counts_df = pd.DataFrame({
    "ORG": pd.Series(all_entity_counts),
    "BETTER_PRICE": pd.Series(all_better_price_counts),
    "FEATURE": pd.Series(all_quality_counts),
    "ADVERTISING": pd.Series(all_advertising_counts)
}).fillna(0).astype(int)



In [ ]:
combined_counts_df[combined_counts_df['BETTER_PRICE']>0].sum()

ORG               0
BETTER_PRICE    149
FEATURE           0
ADVERTISING       0
dtype: int64

In [ ]:
combined_counts_df[combined_counts_df['FEATURE']>1]

,ORG,BETTER_PRICE,FEATURE,ADVERTISING
Quality,0,0,3,0
comfort,0,0,2,0
design,0,0,7,0
fiberglass,0,0,9,0
filter,0,0,15,0
filters,0,0,12,0
fit,0,0,10,0
full face,0,0,7,0
full-face,0,0,2,0
look,0,0,10,0


In [ ]:
combined_counts_df[combined_counts_df['ADVERTISING']>1]

,ORG,BETTER_PRICE,FEATURE,ADVERTISING
Facebook,0,0,0,4
ad,0,0,0,3
information,0,0,0,4
site,0,0,0,5
videos,0,0,0,3
website,0,0,0,4


In [ ]:
Feature=combined_counts_df['FEATURE'].value_counts()

In [ ]:
Feature_top_5=Feature.head(5)

In [ ]:
Feature_top_5

FEATURE
0     55
1     17
2      5
7      3
10     2
Name: count, dtype: int64

In [ ]:
from google.colab import files
comments.to_excel('Competitor.xlsx')
# files.download('Competitor.xlsx')

In [ ]:
whatever

NameError: name 'whatever' is not defined

## Concern


In [ ]:
import pandas as pd
import nltk
import warnings
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.language import Language
from spacy.pipeline import EntityRuler
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
import ast

In [ ]:
warnings.filterwarnings('ignore')

In [ ]:
comments=pd.read_csv('Post_Purchase.csv')

In [ ]:
comments.rename(columns={'If you were considering another brand, who were they and what led you to choosing us instead?': 'Competitor',
                         'Did you have any concerns, doubts, or uncertainties that almost stopped you from purchasing today? If so, what were they? (We love feedback - don’t hold back)': 'Concern',
                         'What ONE factor influenced your decision to purchase the most?': 'Purchase_Influencing_Factor',
                         'What motivated you to purchase from us today? (This helps us a lot - thanks in advance!)':'Purchase_Reason'},inplace=True)

In [ ]:
comments.shape

(1488, 11)

In [ ]:
comments['Concern'].isnull().sum()

751

In [ ]:
comments['Concern'].value_counts()


Concern
No                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        164
no                                                                                                                                                                                                                                                                                                                                                                                                

In [ ]:
a = comments['Concern'].value_counts()
df = a.reset_index()
df.columns = ['Concern', 'Count']

# Sort the DataFrame in descending order
df_sorted = df.sort_values(by='Count', ascending=False)

# Set the pandas option to display all rows
pd.set_option('display.max_rows', None)

In [ ]:
df_sorted

,Concern,Count
0,No,164
1,no,27
2,None,14
3,Nope,11
4,none,9
5,Price,7
6,No,5
7,No.,4
8,NO,4
9,Nothing,3


In [ ]:
# Drop null values
comments = comments.dropna(subset=['Concern'])

# Remove comments that just say "No" or "no"
comments = comments[~comments['Concern'].str.lower().eq('no')]

# Download the VADER lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment intensity analyzer
sid = SentimentIntensityAnalyzer()

# Function to classify the sentiment based on the compound score
def classify_sentiment(compound_score):
    if compound_score >= 0.05:
        return 'Positive'
    else:
        return 'Negative'

# Analyze sentiment for each comment in the 'Concern' column and classify it
comments['compound'] = comments['Concern'].apply(lambda x: sid.polarity_scores(x)['compound'])
comments['sentiment_label'] = comments['compound'].apply(classify_sentiment)



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
comments['sentiment_label'].value_counts()

sentiment_label
Negative    334
Positive    208
Name: count, dtype: int64

In [ ]:
comments.head(50)

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,Purchase_Reason,Purchase_Influencing_Factor,Concern,Competitor,compound,sentiment_label
1,Post Purchase Survey V1,josephdhorn@gmail.com,Joseph,Horn,2024-01-27T00:57:53.335Z,post-checkout,227.10,I want to be prepared for any emergencies and ...,Product description of benefits/features,My main concern is if the respiratory mask wil...,"The mask looks great, and it protects from vir...",0.3612,Positive
2,Post Purchase Survey V1,gypsydrifter.mh@gmail.com,Albert,hunter,2024-01-27T01:17:49.725Z,post-checkout,378.47,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,No reviews,Nope,-0.2960,Negative
7,Post Purchase Survey V1,chris@jescoboatcenter.com,chris,Parrott,2024-01-27T04:19:32.406Z,post-checkout,375.33,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,"More information on fit, forum sites say your ...","Mira, price",0.7668,Positive
11,Post Purchase Survey V1,stuartluke54@gmail.com,Luke,Stuart,2024-01-27T15:54:39.286Z,post-checkout,248.65,My job requires protection against environment...,Other (please specify) (Military discount),"Price, I’m poor. Using these gas masks to evic...",NaN,-0.7003,Negative
12,Post Purchase Survey V1,pete.graziano3@gmail.com,Peter,Graziano,2024-01-27T16:07:25.423Z,post-checkout,248.59,My job requires protection against environment...,Product description of benefits/features,None at the moment. I check back in.,"Price, and individuality",0.0000,Negative
14,Post Purchase Survey V1,info@builtcreative.com,Shane,Poteete,2024-01-27T17:09:35.317Z,post-checkout,560.58,I want to be prepared for any emergencies and ...,Other (please specify) (Promotion from 704 Tac...,"No. 704 Tactical does reviews of products, and...",Other brands were 3M or MIRA. Discount/price w...,-0.2960,Negative
15,Post Purchase Survey V1,vanwish@netpenny.net,Cheryl,Van Allsburg,2024-01-27T19:40:05.501Z,post-checkout,91.91,I want to be prepared for any emergencies and ...,Product reviews and testimonials,I saw a claim online that most fire blankets d...,NaN,-0.6061,Negative
17,Post Purchase Survey V1,koko.capalot247@gmail.com,Night,Chaser,2024-01-27T21:43:05.958Z,post-checkout,125.63,I want to be prepared for any emergencies and ...,Product description of benefits/features,"Shipping times, tomorrow is never guaranteed.",NaN,0.0000,Negative
18,Post Purchase Survey V1,christophermeegan84@gmail.com,Christopher,Meegan,2024-01-27T23:14:03.106Z,post-checkout,458.95,Other (please specify) (Life is dangerous and ...,Other (please specify) (Black Scout Survival),Price but I found a fantastic coupon code that...,Quality and price are some of the best around ...,0.7096,Positive
20,Post Purchase Survey V1,williamladley440@gmail.com,William,Ladley,2024-01-28T00:07:43.211Z,post-checkout,229.66,I want to be prepared for any emergencies and ...,Product description of benefits/features,I’m a firefighter and was weary about putting ...,"MSA im familiar with this brand,ease of filter...",0.2023,Positive


In [ ]:
nlp = spacy.load("en_core_web_sm")


In [ ]:
phrase_matcher = PhraseMatcher(nlp.vocab)


In [ ]:
# Define additional NER categories
feature_phrases = [
    "fit", "Fit", "Size", "size", "fire blanket", "Fire blanket", "FireBlanket",
    "Fire Blanket", "Quality", "quality", "Tinted", "tinted", "amber", "Amber",
    "Sizing", "sizing", "filters", "filter", "Filter", "Filters", "cartridges",
    "lenses", "comfort", "comfortable", "Comfort", "Comfortable", "design",
    "Design", "protect", "Protect", "protection", "Protection", "Spectacle Kit",
    "spectacle kit", "reading glasses"
]

money_phrases = [
    "Price", "Expensive", "price", "expensive", "affordability", "affordable",
    "Affordability", "Affordable", "Discount", "discount", "money", "Money"
]

description_phrases = [
    "Information", "information", "sites", "Sites", "terms", "layman's",
    "Website", "website", "reviews", "Reviews"
]

service_phrases = [
    "Shipping", "shipping", "verifying", "verification", "Verifying",
    "Verification", "timely", "time", "Timely", "Time", "delay", "Delay",
    "Exchange", "exchange", "Return", "return", "shipped", "Shipped"
]

In [ ]:
feature_patterns = [nlp.make_doc(text) for text in feature_phrases]
phrase_matcher.add("FEATURE", None, *feature_patterns)
money_patterns = [nlp.make_doc(text) for text in money_phrases]
phrase_matcher.add("MONEY", None, *money_patterns)
description_patterns = [nlp.make_doc(text) for text in description_phrases]
phrase_matcher.add("INFORMATION", None, *description_patterns)
service_patterns = [nlp.make_doc(text) for text in service_phrases]
phrase_matcher.add("SERVICE", None, *service_patterns)

In [ ]:
@Language.component("custom_ner_component")
def add_custom_ents(doc):
    matches = phrase_matcher(doc)
    spans = []
    for match_id, start, end in matches:
        match_label = nlp.vocab.strings[match_id]
        if match_label == "FEATURE":
            span = Span(doc, start, end, label="FEATURE")
        else:
            span = Span(doc, start, end, label=match_label)
        spans.append(span)

    # Combine with existing entities ensuring no overlap
    existing_ents = list(doc.ents)
    for span in spans:
        if not any(span.start < ent.end and span.end > ent.start for ent in existing_ents):
            existing_ents.append(span)

    # Sort the entities to maintain proper sequence
    doc.ents = sorted(existing_ents, key=lambda ent: ent.start)
    return doc

In [ ]:
nlp.add_pipe("custom_ner_component", after='ner')

<function __main__.add_custom_ents(doc)>

In [ ]:
# Function to find NERs in the competitors column and classify them
def find_ners(text):
    doc = nlp(text)
    FEATURE, MONEY, INFORMATION, SERVICE = [], [], [], []
    for ent in doc.ents:
        if ent.label_ == "FEATURE":
          FEATURE.append((ent.text, ent.label_))
        elif ent.label_ == "MONEY":
          MONEY.append((ent.text, ent.label_))
        elif ent.label_ == "INFORMATION":
          INFORMATION.append((ent.text, ent.label_))
        elif ent.label_ == "SERVICE":
          SERVICE.append((ent.text, ent.label_))
    return FEATURE, MONEY, INFORMATION, SERVICE

In [ ]:
# Apply the function to the competitors column
comments["Feature_entities"], comments["Money_entities"], comments["Information_entities"], comments["Service_entities"] = zip(*comments["Concern"].apply(find_ners))

In [ ]:
comments.head(20)

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,Purchase_Reason,Purchase_Influencing_Factor,Concern,Competitor,compound,sentiment_label,Feature_entities,Money_entities,Information_entities,Service_entities
1,Post Purchase Survey V1,josephdhorn@gmail.com,Joseph,Horn,2024-01-27T00:57:53.335Z,post-checkout,227.10,I want to be prepared for any emergencies and ...,Product description of benefits/features,My main concern is if the respiratory mask wil...,"The mask looks great, and it protects from vir...",0.3612,Positive,"[(fit, FEATURE)]",[],[],[]
2,Post Purchase Survey V1,gypsydrifter.mh@gmail.com,Albert,hunter,2024-01-27T01:17:49.725Z,post-checkout,378.47,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,No reviews,Nope,-0.2960,Negative,[],[],"[(reviews, INFORMATION)]",[]
7,Post Purchase Survey V1,chris@jescoboatcenter.com,chris,Parrott,2024-01-27T04:19:32.406Z,post-checkout,375.33,I want to be prepared for any emergencies and ...,Smooth website and navigation experience,"More information on fit, forum sites say your ...","Mira, price",0.7668,Positive,"[(fit, FEATURE), (fit, FEATURE)]",[],"[(information, INFORMATION), (sites, INFORMATI...",[]
11,Post Purchase Survey V1,stuartluke54@gmail.com,Luke,Stuart,2024-01-27T15:54:39.286Z,post-checkout,248.65,My job requires protection against environment...,Other (please specify) (Military discount),"Price, I’m poor. Using these gas masks to evic...",NaN,-0.7003,Negative,[],"[(Price, MONEY)]",[],[]
12,Post Purchase Survey V1,pete.graziano3@gmail.com,Peter,Graziano,2024-01-27T16:07:25.423Z,post-checkout,248.59,My job requires protection against environment...,Product description of benefits/features,None at the moment. I check back in.,"Price, and individuality",0.0000,Negative,[],[],[],[]
14,Post Purchase Survey V1,info@builtcreative.com,Shane,Poteete,2024-01-27T17:09:35.317Z,post-checkout,560.58,I want to be prepared for any emergencies and ...,Other (please specify) (Promotion from 704 Tac...,"No. 704 Tactical does reviews of products, and...",Other brands were 3M or MIRA. Discount/price w...,-0.2960,Negative,[],[],"[(reviews, INFORMATION)]",[]
15,Post Purchase Survey V1,vanwish@netpenny.net,Cheryl,Van Allsburg,2024-01-27T19:40:05.501Z,post-checkout,91.91,I want to be prepared for any emergencies and ...,Product reviews and testimonials,I saw a claim online that most fire blankets d...,NaN,-0.6061,Negative,[],[],[],[]
17,Post Purchase Survey V1,koko.capalot247@gmail.com,Night,Chaser,2024-01-27T21:43:05.958Z,post-checkout,125.63,I want to be prepared for any emergencies and ...,Product description of benefits/features,"Shipping times, tomorrow is never guaranteed.",NaN,0.0000,Negative,[],[],[],"[(Shipping, SERVICE)]"
18,Post Purchase Survey V1,christophermeegan84@gmail.com,Christopher,Meegan,2024-01-27T23:14:03.106Z,post-checkout,458.95,Other (please specify) (Life is dangerous and ...,Other (please specify) (Black Scout Survival),Price but I found a fantastic coupon code that...,Quality and price are some of the best around ...,0.7096,Positive,[],"[(Price, MONEY)]",[],[]
20,Post Purchase Survey V1,williamladley440@gmail.com,William,Ladley,2024-01-28T00:07:43.211Z,post-checkout,229.66,I want to be prepared for any emergencies and ...,Product description of benefits/features,I’m a firefighter and was weary about putting ...,"MSA im familiar with this brand,ease of filter...",0.2023,Positive,[],"[(discount, MONEY)]",[],"[(verifying, SERVICE)]"


In [ ]:
from google.colab import files
comments.to_excel('Doubts.xlsx')
# files.download('Doubts.xlsx')

In [ ]:
# Function to count unique entities from a list str# Define the function to count rows with non-empty entity lists
def count_unique_entities(ner_list):
    unique_entities = set()
    if isinstance(ner_list, list):  # Ensure the input is a list
        for entity, label in ner_list:
            unique_entities.add(entity.lower())  # Convert entity to lowercase
    else:
        print(f"Row is not a list: {ner_list}")
    return unique_entities

In [ ]:
# def count_entities(ner_list):
#     entity_count = Counter()
#     for entity, label in ner_list:
#        entity_count[entity] += 1
#     return dict(entity_count)

In [ ]:
# Initialize counters for rows with at least one occurrence
from collections import Counter
all_feature_counts = Counter()
all_money_counts = Counter()
all_information_counts = Counter()
all_service_counts = Counter()


# Aggregate the entity counts from all rows for each category
for unique_entities in comments["Feature_entities"].apply(count_unique_entities):
    all_feature_counts.update(unique_entities)
for unique_entities in comments["Money_entities"].apply(count_unique_entities):
    all_money_counts.update(unique_entities)
for unique_entities in comments["Information_entities"].apply(count_unique_entities):
    all_information_counts.update(unique_entities)
for unique_entities in comments["Service_entities"].apply(count_unique_entities):
    all_service_counts.update(unique_entities)


In [ ]:
# Aggregate the entity counts from all rows for each category
# all_feature_counts = Counter()
# all_money_counts = Counter()
# all_information_counts = Counter()
# all_service_counts = Counter()

# for entity_dict in comments["Feature_entities"].apply(count_entities):
#     all_feature_counts.update(entity_dict)
# for entity_dict in comments["Money_entities"].apply(count_entities):
#     all_money_counts.update(entity_dict)
# for entity_dict in comments["Information_entities"].apply(count_entities):
#     all_information_counts.update(entity_dict)
# for entity_dict in comments["Service_entities"].apply(count_entities):
#     all_service_counts.update(entity_dict)

In [ ]:
combined_concern_df = pd.DataFrame({
    "Feature_entities": pd.Series(all_feature_counts),
    "Money_entities": pd.Series(all_money_counts),
    "Information_entities": pd.Series(all_information_counts),
    "Service_entities": pd.Series(all_service_counts)
}).fillna(0).astype(int)

In [ ]:
combined_concern_df

,Feature_entities,Money_entities,Information_entities,Service_entities
$30 to $300,0,1,0,0
0.3 micro,0,1,0,0
20,0,1,0,0
30 dollar,0,1,0,0
500,0,2,0,0
affordability,0,2,0,0
amber,4,0,0,0
cartridges,4,0,0,0
comfortable,2,0,0,0
design,2,0,0,0


In [ ]:
combined_concern_df['Feature_entities'].sum()

176

In [ ]:
combined_concern_df['Money_entities'].sum()

92

In [ ]:
combined_concern_df['Information_entities'].sum()

50

In [ ]:
combined_concern_df['Service_entities'].sum()

33

In [ ]:
feature=combined_concern_df[combined_concern_df['Feature_entities']>0]

In [ ]:
feature.groupby(level=0).sum()

,Feature_entities,Money_entities,Information_entities,Service_entities
amber,4,0,0,0
cartridges,4,0,0,0
comfortable,2,0,0,0
design,2,0,0,0
filter,31,0,0,0
filters,53,0,0,0
fit,18,0,0,0
lenses,6,0,0,0
protect,4,0,0,0
protection,13,0,0,0


In [ ]:
money=combined_concern_df[combined_concern_df['Money_entities']>0]

In [ ]:
money.groupby(level=0).sum()

,Feature_entities,Money_entities,Information_entities,Service_entities
$30 to $300,0,1,0,0
0.3 micro,0,1,0,0
20,0,1,0,0
30 dollar,0,1,0,0
500,0,2,0,0
affordability,0,2,0,0
discount,0,27,0,0
expensive,0,7,0,0
money,0,12,0,0
only $50,0,1,0,0


In [ ]:
information=combined_concern_df[combined_concern_df['Information_entities']>0]

In [ ]:
information.groupby(level=0).sum()

,Feature_entities,Money_entities,Information_entities,Service_entities
information,0,0,10,0
layman's,0,0,1,0
reviews,0,0,22,0
sites,0,0,1,0
terms,0,0,2,0
website,0,0,14,0


In [ ]:
service=combined_concern_df[combined_concern_df['Service_entities']>0]

In [ ]:
service.groupby(level=0).sum()

,Feature_entities,Money_entities,Information_entities,Service_entities
exchange,0,0,0,1
return,0,0,0,5
shipped,0,0,0,2
shipping,0,0,0,7
time,0,0,0,15
timely,0,0,0,1
verification,0,0,0,1
verifying,0,0,0,1


##Purchase Blockers

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.language import Language
from spacy.pipeline import EntityRuler
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()# Visualise inside a notebook
import en_core_web_md
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaMulticore
from gensim.models import CoherenceModel
import warnings
import re

In [ ]:
pb=pd.read_csv('Purchase_Blockers.csv')

In [ ]:
pb.shape

(4559, 10)

In [ ]:
pb.head()

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,Is there anything holding you back from buying this product right now?,"We’re sorry to hear that. What’s the reason that is holding you back from buying? (Please be honest, we love feedback)",Would you mind explaining further? We are trying to constantly improve so we appreciate it!
0,Purchase Blockers,NaN,NaN,NaN,2024-01-29T15:01:50.288Z,on-site,NaN,No,NaN,NaN
1,Purchase Blockers,NaN,NaN,NaN,2024-01-29T15:11:07.425Z,on-site,NaN,Yes,I understood the product benefits but the prod...,Looking for cbrn
2,Purchase Blockers,NaN,NaN,NaN,2024-01-29T15:34:49.259Z,on-site,NaN,No,NaN,NaN
3,Purchase Blockers,NaN,NaN,NaN,2024-01-29T16:37:34.482Z,on-site,NaN,Yes,I don’t have enough information to make an inf...,Needing to know differences between certain ma...
4,Purchase Blockers,NaN,NaN,NaN,2024-01-29T17:12:48.793Z,on-site,NaN,No,NaN,NaN


In [ ]:
pb.rename(columns={'Is there anything holding you back from buying this product right now?':'Purchase Hesitation',
                   'We’re sorry to hear that. What’s the reason that is holding you back from buying? (Please be honest, we love feedback)': 'Hesitation Reason',
                   'Would you mind explaining further? We are trying to constantly improve so we appreciate it!':'Hesitation Description'},inplace=True)

In [ ]:
pb.head()

,surveyName,email,customerFirstName,customerLastName,responseDate,channelName,orderValue,Purchase Hesitation,Hesitation Reason,Hesitation Description
0,Purchase Blockers,NaN,NaN,NaN,2024-01-29T15:01:50.288Z,on-site,NaN,No,NaN,NaN
1,Purchase Blockers,NaN,NaN,NaN,2024-01-29T15:11:07.425Z,on-site,NaN,Yes,I understood the product benefits but the prod...,Looking for cbrn
2,Purchase Blockers,NaN,NaN,NaN,2024-01-29T15:34:49.259Z,on-site,NaN,No,NaN,NaN
3,Purchase Blockers,NaN,NaN,NaN,2024-01-29T16:37:34.482Z,on-site,NaN,Yes,I don’t have enough information to make an inf...,Needing to know differences between certain ma...
4,Purchase Blockers,NaN,NaN,NaN,2024-01-29T17:12:48.793Z,on-site,NaN,No,NaN,NaN


In [ ]:
pb.isnull().sum()

surveyName                   0
email                     4558
customerFirstName         4558
customerLastName          4558
responseDate                 0
channelName                  0
orderValue                4559
Purchase Hesitation          0
Hesitation Reason         2101
Hesitation Description    4011
dtype: int64

In [ ]:
# pb['Hesitation Reason'].value_counts()

In [ ]:
# pb.dropna(subset=('Hesitation Reason'),inplace=True)

In [ ]:
pb['Hesitation Reason'].value_counts()

Hesitation Reason
Something else                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 688
I don’t have enough information to make an informed decision                                                                                                                                                                                                                                                                                                                                                                       

In [ ]:
# pb[pb['Hesitation Reason'].str.contains('money')].value_counts()

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [ ]:
nlp = spacy.load("en_core_web_sm")
phrase_matcher = PhraseMatcher(nlp.vocab)

In [ ]:
# Define the phrases
feature_phrases = [
                 'cbrn','CBRN','Cbrn','filter','Filter','filters',
                 'Filters','fit','fitting','Fit','Fitting','Size',
                 'size','Sizing','sizing','niosh','NIOSH','compatible',
                 'Compatible','protect','Protection','maufacture','Manufacture',
                 'quality','Quality','tear gas','chemical spill','ANSI','ansi',
                 'Ansi rating','chemical','teargas','nuclear','biological','gas',
                 'gases'
                 ]

Information_Deficiency = [
                  "differences", "Differences", "difference", "Difference",
                   "pros", "cons", "learn", "how", "How",
                   "need", "Need", "figure out", "tested", "Tested",
                   "testing", "Testing", "test", "Test",
                   "know", "Know", "IDK", "Looking", "looking",
                   "Not sure", "not sure", "wondering", "Wondering",
                   "unsure", "Unsure", "information", "Information",
                   "explain", "explanation", "What", "what",
                   "reviews", "Reviews", "compare", "comparison",
                   "Unclear", "unclear"
                  ]

In [ ]:
feature_patterns = [nlp.make_doc(text) for text in feature_phrases]
phrase_matcher.add("FEATURE", None, *feature_patterns)
information_patterns = [nlp.make_doc(text) for text in Information_Deficiency]
phrase_matcher.add("INFORMATION_DEFICIENCY", None, *information_patterns)


In [ ]:
from google.colab import files
pb.to_excel("Blockers.xlsx")
files.download('Blockers.xlsx')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
@Language.component("custom_ner_component")
def add_custom_ents(doc):
    matches = phrase_matcher(doc)
    spans = []
    for match_id, start, end in matches:
        match_label = nlp.vocab.strings[match_id]
        if match_label == "FEATURE":
            span = Span(doc, start, end, label="FEATURE")
        else:
            span = Span(doc, start, end, label=match_label)
        spans.append(span)

    # Combine with existing entities ensuring no overlap
    existing_ents = list(doc.ents)
    for span in spans:
        if not any(span.start < ent.end and span.end > ent.start for ent in existing_ents):
            existing_ents.append(span)

    # Sort the entities to maintain proper sequence
    doc.ents = sorted(existing_ents, key=lambda ent: ent.start)
    return doc

In [ ]:
# Add the custom NER component to the pipeline
if "custom_ner_component" in nlp.pipe_names:
    nlp.remove_pipe("custom_ner_component")
nlp.add_pipe("custom_ner_component", after='ner')

<function __main__.add_custom_ents(doc)>

In [ ]:
def find_ners(text):
    if isinstance(text, str):
        doc = nlp(text)
        FEATURE, INFORMATION_DEFICIENCY = [], []
        for ent in doc.ents:
            if ent.label_ == "FEATURE":
                FEATURE.append((ent.text, ent.label_))
            elif ent.label_ == "INFORMATION_DEFICIENCY":
                INFORMATION_DEFICIENCY.append((ent.text, ent.label_))
        return FEATURE, INFORMATION_DEFICIENCY
    else:
        # Return empty lists for non-string inputs
        return [], []

In [ ]:
# Apply the function and handle non-string values
pb["Feature_entities"], pb["Information_entities"] = zip(*pb["Hesitation Description"].apply(find_ners))

# Ensure columns have lists
pb["Feature_entities"] = pb["Feature_entities"].apply(lambda x: x if isinstance(x, list) else [])
pb["Information_entities"] = pb["Information_entities"].apply(lambda x: x if isinstance(x, list) else [])

In [ ]:
def count_unique_entities(ner_list):
    unique_entities = set()
    if isinstance(ner_list, list):  # Ensure the input is a list
        for entity, label in ner_list:
            unique_entities.add(entity.lower())  # Convert entity to lowercase
    else:
        print(f"Row is not a list: {ner_list}")
    return unique_entities

In [ ]:
from collections import Counter
all_feature_counts = Counter()
all_information_counts = Counter()


# Aggregate the entity counts from all rows for each category
for unique_entities in pb["Feature_entities"].apply(count_unique_entities):
    all_feature_counts.update(unique_entities)
for unique_entities in pb["Information_entities"].apply(count_unique_entities):
    all_information_counts.update(unique_entities)


In [ ]:
combined_blockers_df = pd.DataFrame({
    "Feature_entities": pd.Series(all_feature_counts),
    "Information_entities": pd.Series(all_information_counts),
}).fillna(0).astype(int)

In [ ]:
combined_blockers_df[(combined_blockers_df['Information_entities'] > 0)]


,Feature_entities,Information_entities
compare,0,9
comparison,0,4
cons,0,2
difference,0,8
differences,0,3
explain,0,2
figure out,0,7
how,0,46
information,0,7
know,0,55


In [ ]:
combined_blockers_df['Feature_entities'].sum()

291

In [ ]:
combined_blockers_df['Information_entities'].sum()

423